# 抢票脚本 6.0

## 说明：
- 可编辑同目录配置文件(`config.json`) ，也可交互式输入
- 新增可选参数：`target_train_number`（如 `G1234`）。
  - 若设置且非空：优先按照“指定车次”抢票（忽略时间段过滤策略）。
  - 若未设置或为空：按原策略“出发时间范围过滤”，优先点击范围内最早可预订的车次。
- 提前打开 `12306 App` 扫码准备登录


In [7]:
# 导入依赖与通用工具
import json
import os
import re
import time
import random
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.options import Options

CONFIG_PATH = 'config.json'  #配置文件路径

def load_config(path=CONFIG_PATH):
    if os.path.exists(path):
        try:
            with open(path, 'r', encoding='utf-8') as f:
                return json.load(f)
        except Exception as e:
            print(f'读取配置失败，将进入交互输入: {e}')
    return None

def save_config(cfg, path=CONFIG_PATH):
    try:
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(cfg, f, ensure_ascii=False, indent=2)
        print(f'已保存配置到: {os.path.abspath(path)}')
    except Exception as e:
        print(f'保存配置失败: {e}')

def _ask(prompt, default=None):
    try:
        s = input(f"{prompt}{' ['+str(default)+']' if default is not None else ''}: ").strip()
        return s if s else default
    except Exception:
        return default

def _parse_time_range(s):
    # 形如 '07:00-09:30'
    if not s: return None
    m = re.match(r'^(\d{2}:\d{2})\s*-\s*(\d{2}:\d{2})$', s)
    if not m: return None
    return m.group(1), m.group(2)

def _normalize_ttn(v):
    # 将各种“空值表达”统一视为空
    if v is None:
        return ''
    t = str(v).strip()
    if t == '':
        return ''
    t_upper = t.upper()
    if t_upper in {'-', 'NULL', 'NONE', 'N/A', 'NA', '无', '不填写', '空'}:
        return ''
    return t_upper

def get_params(config_path=CONFIG_PATH, allow_save=True):
    cfg = load_config(config_path) or {}

    from_station = cfg.get('from_station') or _ask('出发站(完整名称)', '龙川西')
    to_station = cfg.get('to_station') or _ask('到达站(完整名称)', '河源东')
    travel_date = cfg.get('travel_date') or _ask('出发日期(YYYY-MM-DD)', '2025-10-05')

    # 票型：adult/学生
    ticket_type = (cfg.get('ticket_type') or _ask('票型(adult/student)', 'adult')).strip().lower()
    if ticket_type not in ('adult','student'):
        print('票型输入无效，使用 adult')
        ticket_type = 'adult'

    # 出发时间范围（兼容字符串或字典；保存时保持原始格式）
    tr_cfg = cfg.get('depart_time_range')
    tr_source_type = 'dict' if (isinstance(tr_cfg, dict) and isinstance(tr_cfg.get('start'), str) and isinstance(tr_cfg.get('end'), str)) else 'string'
    tr_str_value = None
    if tr_source_type == 'dict':
        depart_time_range = {'start': tr_cfg['start'], 'end': tr_cfg['end']}
    else:
        tr_str_value = tr_cfg if isinstance(tr_cfg, str) else _ask('出发时间范围(HH:MM-HH:MM)', '07:00-09:30')
        parsed = _parse_time_range(tr_str_value)
        if not parsed:
            print('时间范围格式无效，使用默认 07:00-09:30')
            parsed = ('07:00','09:30')
        depart_time_range = {'start': parsed[0], 'end': parsed[1]}

    # 席别偏好：仅记录，不改变 4.0 的基础流程逻辑
    seat_category = cfg.get('seat_category') or _ask('席别(如 二等座/一等座/商务座)', '二等座')
    seat_position_preference = cfg.get('seat_position_preference') or _ask('选座偏好(first/aisle/window)', 'first')

    # 开售时间。若为空或过去时间则立即开始。
    booking_start_time = cfg.get('booking_start_time') or _ask('抢票开始时间(YYYY-MM-DD HH:MM:SS，可留空)', '')

    # 新增：目标车次（不为空则优先按车次策略）
    target_train_number = _normalize_ttn(cfg.get('target_train_number'))
    if (cfg.get('target_train_number') is None) and (target_train_number == ''):
        # 配置里不存在该键，则提供一次交互输入（可留空）
        target_train_number = _normalize_ttn(_ask('指定车次(如 G1234，可留空)', ''))

    params = {
        'from_station': from_station,
        'to_station': to_station,
        'travel_date': travel_date,
        'ticket_type': ticket_type,
        'depart_time_range': depart_time_range,
        'seat_category': seat_category,
        'seat_position_preference': seat_position_preference,
        'booking_start_time': booking_start_time,
        'target_train_number': target_train_number,
    }

    # 回写配置（保持原字段格式与额外键；不新增 _saved）
    if allow_save:
        save = (_ask('是否保存到配置文件? (y/N)', 'y') in ('y','Y','yes','YES'))
        if save:
            cfg_out = dict(cfg)  # 以原配置为基础，保留未知字段
            cfg_out.update({
                'from_station': from_station,
                'to_station': to_station,
                'travel_date': travel_date,
                'ticket_type': ticket_type,
                'seat_category': seat_category,
                'seat_position_preference': seat_position_preference,
                'booking_start_time': booking_start_time,
                'target_train_number': target_train_number,
            })
            if tr_source_type == 'dict':
                cfg_out['depart_time_range'] = {'start': depart_time_range['start'], 'end': depart_time_range['end']}
            else:
                cfg_out['depart_time_range'] = f"{depart_time_range['start']}-{depart_time_range['end']}"
            save_config(cfg_out, config_path)
    return params

def parse_hhmm_to_minutes(hhmm):
    try:
        h, m = map(int, hhmm.split(':'))
        return h*60 + m
    except Exception:
        return None

def time_in_range(t, start, end):
    # t/start/end 均为 'HH:MM'
    tm = parse_hhmm_to_minutes(t)
    sm = parse_hhmm_to_minutes(start)
    em = parse_hhmm_to_minutes(end)
    if None in (tm, sm, em):
        return False
    return sm <= tm <= em

In [8]:


def select_seat_fast(driver, preferred_type="过道"):
    print(f"快速选择座位，偏好: {preferred_type}")
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'seat-sel-bd'))
        )
    except Exception as e:
        print(f'座位选择对话框加载失败: {e}')
        return False
    try:
        seats = driver.find_elements(By.XPATH, "//div[@class='seat-sel-bd']//a[contains(@href, 'javascript:')]")
        if not seats:
            return False
        # 为保证速度，仍然选择第一个可点击座位
        seats[0].click()
        print('已快速选择一个座位')
        return True
    except Exception as e:
        print(f'快速选座失败: {e}')
        return False


def extract_depart_time_from_row(row):
    """更稳健地从行中提取出发时间(HH:MM)。
    优先从第2列（常为出发时间列）抓取；失败再回退整行匹配。"""
    # 1) 优先从第2列目标元素中找时间
    try:
        cand = row.find_elements(
            By.XPATH,
            ".//td[position()=2 or contains(@class,'cdz') or contains(@class,'cds')]//*[self::strong or self::span or self::div or self::em]"
        )
        for c in cand:
            t = (c.text or '').strip()
            if re.fullmatch(r'([01]\d|2[0-3]):([0-5]\d)', t):
                return t
    except Exception:
        pass

    # 2) 退化为整行的首个时间匹配（可能会匹配到达时间，不推荐但作为兜底）
    try:
        txt = row.text or ''
        m = re.search(r'(?:^|\s)([01]\d|2[0-3]):([0-5]\d)(?:\s|$)', txt)
        if m:
            return f"{m.group(1)}:{m.group(2)}"
    except Exception:
        pass

    return None


def extract_train_number_from_row(row):
    # 提取形如 G1234/D5678/K80 等车次号
    try:
        # 常见位置：第一列 strong/span
        cand = row.find_elements(By.XPATH, ".//td[1]//*[self::strong or self::span or self::a or self::div]")
        for c in cand:
            t = (c.text or '').strip().upper()
            if re.fullmatch(r'[GDKCTZXYFS]\d{1,5}', t):
                return t
        # 兜底：整行文本中匹配
        txt = (row.text or '').upper()
        m = re.search(r'\b([GDKCTZXYFS]\d{1,5})\b', txt)
        if m:
            return m.group(1)
    except Exception:
        pass
    return None


def click_book_in_row(row, driver):
    try:
        btns = row.find_elements(By.XPATH, ".//a[contains(text(),'预订')]")
        if not btns:
            return False
        btn = btns[0]
        driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", btn)
        time.sleep(0.2)
        try:
            btn.click()
            return True
        except Exception:
            driver.execute_script('arguments[0].click();', btn)
            return True
    except Exception as e:
        print(f'点击预订失败: {e}')
        return False


def _find_rows(driver):
    # 更精确筛选真实数据行：排除表头/隐藏
    xpath = "//*[@id='queryLeftTable']/tr[not(contains(@class,'ticket-hd')) and not(contains(@style,'display: none'))]"
    return driver.find_elements(By.XPATH, xpath)


def _find_row_by_train_number(driver, target):
    """优先用精确 XPath 在表格中直接找车次号对应的行，失败再回退逐行解析。"""
    target = (target or '').strip().upper()
    if not target:
        return None
    # 方案1：直接文本匹配 a 标签文字
    try:
        nodes = driver.find_elements(By.XPATH, f"//*[@id='queryLeftTable']//a[normalize-space(text())='{target}']/ancestor::tr[1]")
        for n in nodes:
            if n.is_displayed():
                return n
    except Exception:
        pass
    # 方案2：回退到逐行解析
    try:
        rows = _find_rows(driver)
        for r in rows:
            tn = extract_train_number_from_row(r)
            if tn == target:
                return r
    except Exception:
        pass
    return None


def book_by_time_range(driver, start_hhmm, end_hhmm, max_attempts=30, refresh_interval=(3,6)):
    for attempt in range(1, max_attempts+1):
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'queryLeftTable')))
            rows = _find_rows(driver)
            found_times = []
            candidates = []
            for r in rows:
                dep = extract_depart_time_from_row(r)
                if dep:
                    found_times.append(dep)
                if dep and time_in_range(dep, start_hhmm, end_hhmm):
                    # 只考虑有“预订”按钮的行，提升命中率
                    if r.find_elements(By.XPATH, ".//a[contains(text(),'预订')]"):
                        candidates.append((dep, r))
            if candidates:
                candidates.sort(key=lambda x: parse_hhmm_to_minutes(x[0]))
                dep, row = candidates[0]
                print(f'发现时间匹配的车次: {dep}，尝试预订...')
                if click_book_in_row(row, driver):
                    return f'成功尝试预订出发时间 {dep} 的车次'
            else:
                if attempt == 1 or attempt % 5 == 0:
                    preview = ','.join(sorted(set(found_times))[:6]) if found_times else '无'
                    print(f'本次共扫描 {len(rows)} 行，解析到出发时刻: {preview}；未命中范围 {start_hhmm}-{end_hhmm}')
        except Exception as e:
            print(f'第{attempt}次尝试失败: {e}')
        # 刷新查询
        if attempt < max_attempts:
            try:
                refresh_btn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'query_ticket')))
                refresh_btn.click()
            except Exception as e:
                print(f'点击查询按钮刷新失败: {e}，尝试整页刷新')
                driver.refresh()
            wait_time = random.uniform(*refresh_interval)
            print(f'无匹配结果，等待{wait_time:.2f}s后重试...')
            time.sleep(wait_time)
    return '没抢到，可惜~'


def book_by_train_number(driver, target_train_number, max_attempts=30, refresh_interval=(2,4)):
    target = (target_train_number or '').strip().upper()
    if not target:
        return '未设置目标车次'
    for attempt in range(1, max_attempts+1):
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'queryLeftTable')))
            row = _find_row_by_train_number(driver, target)
            if row is not None:
                print(f'发现目标车次 {target}，尝试预订...')
                if click_book_in_row(row, driver):
                    return f'成功尝试预订指定车次 {target}'
        except Exception as e:
            print(f'第{attempt}次尝试失败: {e}')
        # 刷新查询
        if attempt < max_attempts:
            try:
                refresh_btn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'query_ticket')))
                refresh_btn.click()
            except Exception as e:
                print(f'点击查询按钮刷新失败: {e}，尝试整页刷新')
                driver.refresh()
            wait_time = random.uniform(*refresh_interval)
            print(f'未出现目标车次 {target}，等待{wait_time:.2f}s后重试...')
            time.sleep(wait_time)
    return f'未抢到指定车次 {target}，可惜~'

In [ ]:


def main():
    params = get_params(CONFIG_PATH)
    print('当前任务参数:')
    print(json.dumps(params, ensure_ascii=False, indent=2))

    edge_options = Options()
    edge_options.add_experimental_option('detach', True)
    edge_options.add_argument('--disable-blink-features=AutomationControlled')
    edge_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.3485.54')
    driver = webdriver.Edge(options=edge_options)
    driver.get('https://www.12306.cn')
    driver.maximize_window()

    # 登录（保持原点击，但下调等待）
    try:
        login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'J-btn-login')))
        login_button.click()
    except Exception as e:
        print(f'点击登录按钮失败：{e}')
        driver.quit(); raise SystemExit

    try:
        scan_login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[text()='扫码登录']")))
        scan_login_button.click()
        print('已切换到扫码登录，请用手机12306扫码...')
    except Exception as e:
        print(f'点击扫码登录按钮失败：{e}')
        driver.quit(); raise SystemExit

    # 等待登录成功（更短轮询）
    login_success = False
    print('等待扫码登录...')
    for _ in range(40):
        try:
            WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, "//a[text()='个人中心']")))
            login_success = True
            break
        except Exception:
            time.sleep(0.25)
    if not login_success:
        print('登录超时或未成功，请重新尝试')
        driver.quit(); raise SystemExit

    # 进入购票页面
    try:
        ticket_link = WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.ID, 'link_for_ticket')))
        ticket_link.click()
        # 若打开了新标签页，切换到最新窗口
        time.sleep(0.2)
        try:
            if len(driver.window_handles) > 1:
                driver.switch_to.window(driver.window_handles[-1])
        except Exception:
            pass
        print('已进入购票页面')
    except Exception as e:
        print(f'进入购票页面失败：{e}')
        driver.quit(); raise SystemExit

    # 出发/到达站（仅缩短等待，不改变逻辑）
    try:
        from_station_input = WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.ID, 'fromStationText')))
        from_station_input.click(); from_station_input.clear(); from_station_input.send_keys(params['from_station'])
        print(f"已输入出发地: {params['from_station']}")
        first_option = WebDriverWait(driver, 6).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#citem_0 > span:nth-child(1)')))
        first_option.click()
    except Exception as e:
        print(f'操作出发地输入框失败：{e}')
        driver.quit(); raise SystemExit

    try:
        to_station_input = WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.ID, 'toStationText')))
        to_station_input.click(); to_station_input.clear(); to_station_input.send_keys(params['to_station'])
        print(f"已输入目的地: {params['to_station']}")
        first_option = WebDriverWait(driver, 6).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#citem_0 > span:nth-child(1)')))
        first_option.click()
    except Exception as e:
        print(f'操作目的地输入框失败：{e}')
        driver.quit(); raise SystemExit

    # 出发日期
    try:
        date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'train_date')))
        date_input.click(); date_input.clear(); date_input.send_keys(params['travel_date'])
        print(f"已输入出发时间: {params['travel_date']}")
        try:
            driver.find_element(By.CLASS_NAME, 'cal').click()
        except Exception:
            pass
    except Exception as e:
        print(f'时间输入框操作失败：{e}')
        driver.quit(); raise SystemExit

    # 票型
    try:
        if params['ticket_type'] == 'student':
            WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.ID, 'sf2'))).click(); print('已选择学生票')
        else:
            WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.ID, 'sf1'))).click(); print('已选择成人票')
    except Exception as e:
        print(f'票种选择失败：{e}')
        driver.quit(); raise SystemExit

    # 开售时间等待（缩短轮询）
    try:
        bst = (params.get('booking_start_time') or '').strip()
        if bst:
            start_datetime = datetime.strptime(bst, '%Y-%m-%d %H:%M:%S')
            now = datetime.now()
            if now < start_datetime:
                wait_seconds = (start_datetime - now).total_seconds()
                if wait_seconds > 10:
                    time.sleep(max(0, wait_seconds - 10))
                while datetime.now() < start_datetime:
                    time.sleep(0.05)
        print('到达抢票时间，开始抢票！')
    except Exception as e:
        print(f'时间处理出错: {e}')
        driver.quit(); raise SystemExit

    # 第一次查询（缩短停顿）
    try:
        query_button = WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.ID, 'query_ticket')))
        query_button.click()
        print('已提交查询，正在等待结果...')
        time.sleep(0.2)
    except Exception as e:
        print(f'查询失败：{e}')
        driver.quit(); raise SystemExit

    # 策略：优先按车次号；若未设置车次号则按时间范围
    ttn = (params.get('target_train_number') or '').strip().upper()
    if ttn:
        result_msg = book_by_train_number(driver, ttn, max_attempts=30, refresh_interval=(2,4))
    else:
        tr = params['depart_time_range']
        result_msg = book_by_time_range(driver, tr['start'], tr['end'], max_attempts=30, refresh_interval=(2,4))
    print(result_msg)

    # 后续下单流程（下调等待）
    try:
        passenger_checkbox = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'normalPassenger_0')))
        passenger_checkbox.click(); print('已成功选择乘车人')
    except Exception as e:
        print(f'选择乘车人失败：{e}')
    try:
        WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.ID, 'dialog_xsertcj_ok'))).click()
    except Exception as e:
        print(f'点击确认按钮失败：{e}')

    # 订单页票种选择（成人票）
    try:
        if params['ticket_type'] == 'adult':
            from selenium.webdriver.support.ui import Select
            ticket_type_select = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.ID, 'ticketType_1')))
            Select(ticket_type_select).select_by_value('1')
            print('订单页已选择票种：成人票')
    except Exception as e:
        print(f'订单页选择票种失败：{e}')

    # 提交订单
    try:
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'submitOrder_id'))).click(); print('已成功点击提交订单按钮')
    except Exception as e:
        print(f'点击提交订单按钮失败：{e}')
    time.sleep(0.4)

    # 学生票提示
    if params['ticket_type'] == 'student':
        try:
            WebDriverWait(driver, 6).until(EC.element_to_be_clickable((By.ID, 'qd_closeDefaultWarningWindowDialog_id'))).click()
        except Exception as e:
            print(f'点击确认按钮失败：{e}')

    # 选座（快速）
    select_seat_fast(driver, preferred_type=params.get('seat_position_preference','first'))
    time.sleep(0.8)

    # 最终确认
    try:
        WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.ID, 'qr_submit_id'))).click(); print('已提交最终确认')
    except Exception as e:
        print(f'点击确认按钮失败：{e}')


# 在 Notebook 中直接运行此单元即可启动主流程
if __name__ == '__main__':
    main()